Based on the information provided in the "Optym-Take-Home-Exam.pdf" document, we can develop a Mixed Integer Linear Programming (MILP) model for the Aircraft Route Optimization Problem. The model will include complete notations for input parameters, decision variables, constraints, and the objective function.

# Input Parameters:


1. <b>D</b>: Set of demands (customer flying requests).
2. <b>A</b>: Set of aircraft.
3. <b>L</b>: Set of flight legs.
4. <b>T</b>: Time periods for planning (e.g., days).
5. <b>C<sub>l</sub></b>: Operating cost for each flight leg <i>l ∈ L</i>.
6. <b>R<sub>d</sub></b>: Revenue from fulfilling demand <i>d ∈ D</i>.
7. <b>M</b>: Large positive number (for modeling purposes).
8. <b>S<sub>la</sub></b>: Starting location of aircraft <i>a ∈ A</i> for leg <i>l ∈ L</i>.
9. <b>E<sub>la</sub></b>: Ending location of aircraft <i>a ∈ A</i> for leg <i>l ∈ L</i>.
10. <b>HOS<sub>a</sub></b>: Maximum hours of service for aircraft <i>a ∈ A</i>.
11. <b>Rest<sub>a</sub></b>: Mandatory rest period for aircraft <i>a ∈ A</i> after <i>HOS<sub>a</sub></i> is reached.




# Decision Variables:

1.	x_lat: Binary variable, 1 if aircraft a flies leg l at time t, 0 otherwise.
2.	y_dat: Binary variable, 1 if demand d is fulfilled at time t, 0 otherwise.
3.	z_la: Binary variable, 1 if leg l is flown empty by aircraft a, 0 otherwise.


# Objective Function:

Maximize total profit, which is the revenue from fulfilling demands minus the operating costs:


$$
\text{Maximize } Z = \sum_d \sum_t R_d \cdot y_{dat} - \sum_l \sum_a \sum_t C_l \cdot x_{lat}
$$


## import libraries

In [16]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd
import pickle

## load data

In [17]:
aircraft_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Aircraft.csv")
airports_data = pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Airports.csv", encoding='ISO-8859-1')
demands_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Demands.csv")
distance_time_file =  pd.read_csv("C:/Users/mrkha/OneDrive/Desktop/OPTYM/code/Distance and Flying Time.csv")

# A: Focus on a single day: 24th July

In [18]:
specific_date = pd.Timestamp('2022-07-24')
demands_file['ScheduledDepDatetime'] = pd.to_datetime(demands_file['ScheduledDepDatetime'])
demands_file = demands_file[demands_file['ScheduledDepDatetime'].dt.date == specific_date.date()]

## preprocess

In [19]:
aircraft_set = aircraft_data['AircraftID'].unique()
demands_set = demands_file['DemandID'].unique()
airports_set = airports_data['Airport Name'].unique()  # Use the correct column name
L = list(set(zip(distance_time_file['DepAirport'], distance_time_file['ArrAirport'])))
time_periods = [specific_date.date()]  # One day planning

input size is so big so here is how to reduce and filter the ac set and L

In [20]:
# Filtering aircraft_set based on initial location
# Assuming 'InitialLocation' is a column in aircraft_data
preferred_locations = {'Location1', 'Location2'}  # Replace with actual locations of interest
filtered_aircraft_set = [a for a in aircraft_set if aircraft_data[aircraft_data['AircraftID'] == a]['InitialLocation'].iloc[0] in preferred_locations]

# Filtering L based on specific airport pairs
# Assuming distance_time_data contains relevant airport pairs
relevant_airport_pairs = set(zip(distance_time_file['DepAirport'], distance_time_file['ArrAirport']))  # You can further filter this set as needed
filtered_L = [(dep, arr) for (dep, arr) in L if (dep, arr) in relevant_airport_pairs]


## parameters

In [21]:
fixed_cost_per_leg = 10000000
# Operating cost for each flight leg (assumed to be in Distance and Flying Time data)
operating_costs = {row['DepAirport'] + row['ArrAirport']: row['FlyingTime']
                   for _, row in distance_time_file.iterrows()}
#
# Revenue from fulfilling demand (simplification: proportional to PaxCount)
revenue_per_demand = {row['DemandID']: row['PaxCount'] * 100  # Arbitrary revenue per passenger
                      for _, row in demands_file.iterrows()}

## Model initialization

In [22]:
model = LpProblem("Aircraft_Route_Optimization", LpMaximize)

## Decision Variables

In [23]:
y_vars = LpVariable.dicts("y", [(d, t) for d in demands_set for t in time_periods], cat=LpBinary)

In [24]:
x_vars = LpVariable.dicts("x", 
                          [(a, l, t) for a in filtered_aircraft_set for l in filtered_L for t in time_periods], 
                          cat=LpBinary)

In [25]:
z_vars = LpVariable.dicts("z", 
                          [(a, l) for a in filtered_aircraft_set for l in filtered_L], 
                          cat=LpBinary)

In [26]:
# x_vars = LpVariable.dicts("x", [(a, l, t) for a in aircraft_set for l in L for t in time_periods], cat=LpBinary)
# y_vars = LpVariable.dicts("y", [(d, t) for d in demands_set for t in time_periods], cat=LpBinary)
# z_vars = LpVariable.dicts("z", [(a, l) for a in aircraft_set for l in L], cat=LpBinary)

In [27]:
# # Assumption: Define a set of feasible flight legs
# # This is an illustrative example. Replace with your actual feasibility criteria
# feasible_legs_set = set(zip(distance_time_file['DepAirport'], distance_time_file['ArrAirport']))

# # Define decision variables more efficiently
# x_vars = {}
# for a in aircraft_set:
#     for (dep, arr) in L:
#         # Create variables only for feasible flight legs
#         if (dep, arr) in feasible_legs_set:
#             for t in time_periods:
#                 x_vars[(a, (dep, arr), t)] = LpVariable(f"x_{a}_{dep}_{arr}_{t}", cat=LpBinary)

# y_vars = { (d, t): LpVariable(f"y_{d}_{t}", cat=LpBinary) for d in demands_set for t in time_periods }
# z_vars = { (a, l): LpVariable(f"z_{a}_{l}", cat=LpBinary) for a in aircraft_set for l in L }



## Objective Function

In [28]:
# # Revenue from fulfilling demands
# revenue = lpSum(revenue_per_demand[d] * y_vars[d, t] for d in demands_set for t in time_periods)

# # Operating costs for flying legs
# cost = lpSum(operating_costs[l] * x_vars[a, l, t] for a in aircraft_set for l in L for t in time_periods)

# # Objective function: Maximize profit (Revenue - Cost)
# model += revenue - cost


# Constraints

1. -	Demand Fulfillment: Each demand must be fulfilled at least once in the planning period.


$$\sum_{t} y_{dt} \geq 1 \quad \text{for all } d$$


In [29]:
# Demand Fulfillment Constraint
for d in demands_set:
    model += lpSum(y_vars[d, t] for t in time_periods) >= 1, f"Demand_Fulfillment_{d}"


2. -	Aircraft Route Continuity: Ensures that for each aircraft, the number of arrivals at an airport equals the number of departures.

$$\sum_{l, S_{la} = k} X_{lat} = \sum_{l, E_{la} = k} X_{lat} \quad \text{for all } a, t, \text{ and } k \text{ in airports}$$


In [30]:
for a in filtered_aircraft_set:
    for k in airports_set:  # Assuming this is the set of all airports
        for t in time_periods:
            arrivals = lpSum(x_vars[a, (dep, arr), t] for (dep, arr) in filtered_L if arr == k)
            departures = lpSum(x_vars[a, (dep, arr), t] for (dep, arr) in filtered_L if dep == k)
            model += (arrivals == departures), f"Aircraft_Route_Continuity_{a}_{k}_{t}"


3-	Aircraft Utilization Limit: Each aircraft cannot exceed its maximum hours of service.

$$\sum_{l} \sum_{t} x_{lat} \leq \text{HOS}_a \quad \text{for all } a$$


In [31]:
# Assuming 'InitialHOS' is a column in aircraft_data representing the max hours of service
for a in filtered_aircraft_set:
    max_hours_of_service = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialHOS'].iloc[0]
    model += lpSum(x_vars[a, l, t] for l in filtered_L for t in time_periods) <= max_hours_of_service, f"Aircraft_Utilization_Limit_{a}"


4. 	Mandatory Rest: After reaching the maximum HOS, the aircraft must observe a mandatory rest period.

$$
\begin{aligned}
&\text{If } \quad l \in L, \quad \sum x_{lat} = \text{HOS}_a, \text{ then} \\
&\quad l \in L, \quad \sum x_{la(t+1)} = 0
\end{aligned}
$$


Given the complexity of directly implementing this as a linear constraint, a practical approach might be to set a utilization limit slightly less than the maximum HOS for each day, thereby implicitly allowing for rest time. This approach simplifies the model while achieving the intended outcome of ensuring rest periods.

In [32]:
# Simplified Mandatory Rest Period Constraint
for a in filtered_aircraft_set:
    for t in time_periods[:-1]:  # Exclude the last time period
        max_hours_before_rest = aircraft_data.loc[aircraft_data['AircraftID'] == a, 'InitialHOS'].iloc[0] - 1  # Example: 1 hour less than HOS
        model += lpSum(x_vars[a, l, t] for l in L) <= max_hours_before_rest, f"Mandatory_Rest_{a}_{t}"


5.	Flight Leg Assignment: A flight leg can only be assigned if it is either loaded or flown empty.

$$x_{lat} \leq M \cdot z_{la} \quad \text{for all } l, a, \text{ and } t$$


In [33]:
# Flight Leg Assignment Constraint
M = 10000  # Large positive number for big-M method
for a in filtered_aircraft_set:
    for l in L:
        for t in time_periods:
            model += x_vars[a, l, t] <= M * z_vars[a, l], f"Flight_Leg_Assignment_{a}_{l}_{t}"


## solve the model

In [34]:
model.solve()

1

In [35]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [36]:
# # Retrieve and format the output if the solution is optimal
# if LpStatus[model.status] == 'Optimal':
#     # Example: Print the values of decision variables
#     for var in model.variables():
#         if var.varValue > 0:  # Only print non-zero values
#             print(f"{var.name} = {var.varValue}")

TypeError: '>' not supported between instances of 'NoneType' and 'int'

output csv

In [37]:
import pandas as pd

# Check if the model has an optimal solution
if LpStatus[model.status] == 'Optimal':
    # Initialize a list to store the output data
    output_data = []

    # Iterate over the x_vars to get the flight legs
    for (a, (dep, arr), t), var in x_vars.items():
        if var.varValue > 0:  # Check if the flight leg is used
            # Create a dictionary for the row
            row = {
                "ScenarioID": "BaseScenario",  # Example scenario name
                "AircraftID": a,
                "FlightLegSeqNumber": None,  # To be determined how to calculate
                "DepAirport": dep,
                "DepTime": t  # Assuming 't' is the departure time
            }
            output_data.append(row)

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Sort and potentially modify the DataFrame as needed to match the schema fully
    # ...

    # Export to CSV
    output_df.to_csv("output_flight_legs.csv", index=False)

    print("Output saved to 'output_flight_legs.csv'")
else:
    print("No optimal solution found.")


Output saved to 'output_flight_legs.csv'
